In [9]:
import cv2

In [10]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
age_net = cv2.dnn.readNetFromCaffe('age_deploy.prototxt', 'age_net.caffemodel')
gender_net = cv2.dnn.readNetFromCaffe('gender_deploy.prototxt', 'gender_net.caffemodel')

In [11]:
def detect_gender_and_age(face):
    blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
    age_net.setInput(blob)
    gender_net.setInput(blob)
    age_preds = age_net.forward()
    gender_preds = gender_net.forward()
    age = age_preds[0].argmax() * 5
    gender = "Male" if gender_preds[0][0] > gender_preds[0][1] else "Female"
    return age, gender

In [12]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    for (x, y, w, h) in faces:
        face = frame[y:y + h, x:x + w]
        age, gender = detect_gender_and_age(face)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        label = f"Age: {age}, Gender: {gender}"
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    cv2.imshow("Gender and Age Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [13]:
cap.release()
cv2.destroyAllWindows()